In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

from sklearn.model_selection import train_test_split

# Neuer Abschnitt

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
print(dataset['sex'].unique(), dataset['smoker'].unique(), dataset['region'].unique())

In [ ]:
dataset['sex_num']=np.zeros(len(dataset))
dataset['smoker_num']=np.zeros(len(dataset))
dataset['region_num']=np.zeros(len(dataset))

print(dataset.info())


for person in dataset.index:
  #print(person)
  dataset['sex_num'][person] = np.where(dataset['sex'][person] == 'female', 1, 0)
  dataset['smoker_num'][person] = np.where(dataset['smoker'][person] == 'yes', 1, 0)

  conditions  = [dataset['region'][person] == 'southwest', dataset['region'][person] == 'southeast', dataset['region'][person] == 'northwest', dataset['region'][person] == 'northeast']
  choices     = [0,1,2,3]
  dataset['region_num'][person] = np.select(conditions, choices, default=np.nan)


In [ ]:
# Angenommen, df ist der DataFrame mit den Eingangsdaten
features_list = ['age', 'bmi', 'children', 'sex_num', 'smoker_num', 'region_num']

features = dataset[features_list]
labels = dataset.pop('expenses')


# Aufteilen der Daten in Trainings- und Validierungsdaten
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

In [ ]:
# Define the model architecture
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[6]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
])

# Compile the model with the desired loss function and metrics
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(0.001),
    loss='mse',
    metrics=['mae', 'mse']
)

In [ ]:
# Fit the model to the data
history = model.fit(
    x_train,
    y_train,
    epochs=1000,
    verbose=0
)

In [ ]:
# Validieren des Modells
y_pred = model.predict(x_test).flatten()

# Plot predictions.
a = plt.axes(aspect='equal')
plt.scatter(y_test, y_pred)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(x_test, y_test, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
